## Notebook detailing finding area of cells
Here, we utilize the colorize function in the preprocess module to label the cells. We begin by importing the necessary functions.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
from skimage import io
from skimage.measure import regionprops, regionprops_table
import pandas as pd
sys.path.insert(0, '~/cells-in-gel/cells_in_gel/')

from cells_in_gel import preprocess
from skimage.util import img_as_float, img_as_ubyte
from skimage.transform import rescale

We then import the pictures of cells. For today I am using two z-stacks of cells in the same condition:

In [ ]:
cells = io.imread('C3-NTG-CFbs_NTG1ECM_1mMRGD_20x_002.tif')
cells2 = io.imread('C3-NTG-CFbs_NTG5ECM_1mMRGD_20x_003.tif')

In this next step, we call the function colorize with the specific image, the size of the binarization element, and the size of the image in microns as determined with imageJ or knowledge of the microscope. I then generate a data frame with the label overlays made in colorize, which I included valuable information, such as area, major axis length and eccentricity. 

*On an aside, I update colorize to resize the image so that the regionprops table gives us the area of the cells in \\( micron^2\\). 

In [ ]:
cells_labelled = preprocess.colorize(cells[1], 3, 665.6)
props = ('area', 'major_axis_length', 'minor_axis_length', 'mean_intensity', 'eccentricity', 'slice')
regions = pd.DataFrame(regionprops_table(cells_labelled, rescale(cells[1],0.65), props))
new_dataframe = regions.drop(regions[regions['area'] < 20].index)


I also dropped rows of the dataframe that had small area values, as these are probably artifacts/noise. However, Jenny has been working on this.

In [ ]:
new_dataframe


I do this again with another picture, with the same condition.

In [ ]:
cells_labelled2 = preprocess.colorize(cells2[1], 3, 665.6)
regions2 = pd.DataFrame(regionprops_table(cells_labelled2, rescale(cells2[1],0.65), props))
new_dataframe2 = regions2.drop(regions2[regions2['area'] < 20].index)
new_dataframe2

I then concatenated the two dataframes so I could analyze them together.

In [ ]:
dataframefinal = pd.concat([new_dataframe, new_dataframe2])
dataframefinal = dataframefinal.reset_index()
dataframefinal

Then, I plotted a histogram of the areas.

In [ ]:
dataframefinal.hist(column = 'area')